In [ ]:

from multiprocessing import Pool
import datetime
import json
import re
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os


def complete_links(link):
    if (link.startswith("/") and not link.startswith("//")):
        return "https://www.aznude.com" + link

    elif (link.startswith("//")):
        return "https:" + link

    elif link.startswith("https") or link.startswith("http"):
        return link


def index_to_file_path(index):
    return os.path.join('.', 'updated_pages', f'actress_portfolio_list_{index}.json')


def index_to_save_file_path(index):
    return os.path.join('.', 'updated_pages', f'actress_portfolio_list_{index}.json')


def get_media_file_link_and_tags(video_url):
    response = requests.get(video_url)
    video_soup = BeautifulSoup(response.text, 'html.parser')

    actress_video_tag_list = []
    tag_error = ""
    link_error = ""

    try:
        actress_video_h2_element = video_soup.find_all(
            'h2', class_="video-tags")[0]
        actress_video_tag_element_list = actress_video_h2_element.find_all('a')
        for tag in actress_video_tag_element_list:
            actress_video_tag_list.append({
                "name": tag.text.strip(),
                "link": complete_links(tag.get('href'))
            })
    except Exception as e:
        tag_error = str(e)

    download_link = ""

    try:
        download_div = video_soup.find_all('div', class_='videoButtons')
        for div in download_div:
            if div.text == 'Download':
                download_link = complete_links(div.parent.get('href'))
    except Exception as e:
        link_error = str(e)

    return {
        "tag_list": actress_video_tag_list,
        "media_file_link": download_link,
        "tag_error": tag_error,
        "link_error": link_error
    }


def get_all_actress_portfolio_file():
    folder_path = os.path.join('.', 'actress_portfolio')
    try:
        files = os.listdir(folder_path)
        return files
    except FileNotFoundError:
        print(f"Error: The folder '{folder_path}' does not exist.")
        return []


def read_json_file(file_path):
    """
    Reads a JSON file and returns its content as a Python dictionary.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        dict: The content of the JSON file as a dictionary.
        None: If the file cannot be opened or the JSON data is invalid.
    """

    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error reading JSON file: {e}")
        return None


def add_meadia_file_link_and_tags_to_page(index):
    portfolio_page_file_path = index_to_file_path(index)
    page_content = read_json_file(portfolio_page_file_path)
    for actress in page_content:
        print(actress["actress_name"])
        production_media_list = actress["production_media_list"]
        for production in production_media_list:
            print("    " + production["title"])
            media_list = production["media_list"]
            for media in media_list:
                if media["type"] == "video":
                    link_to_media = complete_links(media["link_to_media"])
                    eid = complete_links(media["eid"])
                    image_source = complete_links(media["image_source"])
                    # link_and_tags = get_media_file_link_and_tags(video_url=link_to_media)

                    media["link_to_media"] = link_to_media
                    media["eid"] = eid
                    media["image_source"] = image_source
                    tag_list = media["tag_list"]
                    for tag in tag_list:
                        tag["link"] = complete_links(tag["link"])
                    # media["tag_list"] = link_and_tags["tag_list"]
                    # media["media_file_link"] = link_and_tags["media_file_link"]
                    # media["tag_error"] = link_and_tags["tag_error"]
                    # media["link_error"] = link_and_tags["link_error"]
            # break
        # break
    return page_content

# page_content = add_meadia_file_link_and_tags_to_page(1)

def save_updated_page(index):
    page_content = add_meadia_file_link_and_tags_to_page(index)
    json_string = json.dumps(page_content)
    with open(index_to_save_file_path(index), 'w') as json_file:
        json_file.write(json_string)


save_updated_page(1)

In [ ]:
from multiprocessing import Pool
import datetime
import json
import re
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os


def get_media_file_link_and_tags(video_url):
    response = requests.get(video_url)
    video_soup = BeautifulSoup(response.text, 'html.parser')

    actress_video_tag_list = []
    tag_error = ""
    link_error = ""

    try:
        actress_video_h2_element = video_soup.find_all(
            'h2', class_="video-tags")[0]
        actress_video_tag_element_list = actress_video_h2_element.find_all('a')
        for tag in actress_video_tag_element_list:
            actress_video_tag_list.append({
                "name": tag.text.strip(),
                "link": tag.get('href')
            })
    except Exception as e:
        tag_error = str(e)

    download_link = ""

    try:
        download_div = video_soup.find_all('div', class_='videoButtons')
        for div in download_div:
            if div.text == 'Download':
                download_link = "https:" + div.parent.get('href')
    except Exception as e:
        link_error = str(e)

    return {
        "tag_list": actress_video_tag_list,
        "media_file_link": download_link,
        "tag_error": tag_error,
        "link_error": link_error
    }


def get_all_actress_portfolio_file():
    folder_path = os.path.join('.', 'actress_portfolio')
    try:
        files = os.listdir(folder_path)
        return files
    except FileNotFoundError:
        print(f"Error: The folder '{folder_path}' does not exist.")
        return []
    

def read_json_file(file_path):
    """
    Reads a JSON file and returns its content as a Python dictionary.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        dict: The content of the JSON file as a dictionary.
        None: If the file cannot be opened or the JSON data is invalid.
    """

    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error reading JSON file: {e}")
        return None


def add_meadia_file_link_and_tags_to_page(portfolio_page_file_path):
    page_content = read_json_file(portfolio_page_file_path)
    for actress in page_content:
        print(actress["actress_name"])
        production_media_list = actress["production_media_list"]
        for production in production_media_list:
            print("    " + production["title"])
            media_list = production["media_list"]
            for media in media_list:
                if media["type"] == "video":
                    link_to_media = "https://www.aznude.com" + media["link_to_media"]
                    # eid = "https://www.aznude.com" + media["eid"]
                    # image_source = "https:" + media["image_source"]
                    link_and_tags = get_media_file_link_and_tags(video_url=link_to_media)

                    # media["link_to_media"] = link_to_media
                    # media["eid"] = eid
                    # media["image_source"] = image_source
                    media["tag_list"] = link_and_tags["tag_list"]
                    media["media_file_link"] = link_and_tags["media_file_link"]
                    media["tag_error"] = link_and_tags["tag_error"]
                    media["link_error"] = link_and_tags["link_error"]
                    # print(media["link_to_media"])
        break
    return page_content


first_page_content = add_meadia_file_link_and_tags_to_page(os.path.join(
    '.', 'actress_portfolio', 'actress_portfolio_list_2510.json'))

In [ ]:
def complete_links(link):
    if (link.startswith("/") and not link.startswith("//")):
        return "https://www.aznude.com" + link

    elif (link.startswith("//")):
        return "https:" + link

    elif link.startswith("https") or link.startswith("http"):
        return link
    

complete_links(
    "https://cdn2.aznude.com/44c150c0fc144b53b09121bea062326f/44c150c0fc144b53b09121bea062326f.jpg")

In [ ]:
from multiprocessing import Pool
import json
import re
import requests
from bs4 import BeautifulSoup

def get_production_info(production):
    h4_element = production.find('h4')
    type = h4_element.span.text.strip().lower().replace(":", "").capitalize()
    title = h4_element.a.text.strip()
    release_info = h4_element.contents[-1].strip().strip('()')

    ongoing_pattern = re.compile(r'(\d{4})-')  # Matches "YYYY-"
    range_pattern = re.compile(r'(\d{4})-(\d{4})')  # Matches "YYYY-YYYY"
    year_pattern = re.compile(r'^(\d{4})$')  # Matches "YYYY"

    ongoing_match = ongoing_pattern.search(release_info)
    range_match = range_pattern.search(release_info)
    year_match = year_pattern.search(release_info)

    if range_match:
        start_year = range_match.group(1)
        end_year = range_match.group(2)
        ongoing = False
    elif ongoing_match:
        start_year = ongoing_match.group(1)
        end_year = None
        ongoing = True
    elif year_match:
        start_year = year_match.group(1)
        end_year = None
        ongoing = False
    else:
        start_year = None
        end_year = None
        ongoing = False

    result = {
        "type": type,
        "title": title,
        "start_year": start_year,
        "end_year": end_year,
        "ongoing": ongoing,
        "media_list": []
    }
    return result


def get_media_info(media):
    if (not media):
        return None
    if media.find('a', class_='video'):
        a_tag = media.find('a')
        href = a_tag.get('href')
        eid = a_tag.get('eid')
        img_src = a_tag.find('img')['src']
        result = {
            "link_to_media": href,
            "eid": eid,
            "image_source": img_src,
            "type": "video"
        }
        return result
    elif media.find('a', class_='picture'):
        a_tag = media.find('a')
        href = a_tag.get('href')
        img_src = a_tag.find('img')['src']
        result = {
            "link_to_media": href,
            "eid": None,
            "image_source": img_src,
            "type": "picture"
        }
        return result


def get_actress_portfolio(url, view_count):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    actress_name = soup.find('h1').text.strip()
    actress_name = re.sub(r'#.*', '', actress_name).strip()
    try:
        birthplace = soup.find(
            'div', class_='banner-info').find('a').text.strip()
    except:
        birthplace = ""
    # birthplace = soup.find('div', class_='banner-info').find('a').text.strip()
    hashtag = soup.find(
        'span', class_='tag-desktop').text.strip().replace("#", "")
    celeb_img_url = soup.find(
        'img', class_="img-circle pull-right img-responsive celeb-img").get('src')

    actress_portfolio = {
        "actress_name": actress_name,
        "birthplace": birthplace,
        "hashtag": hashtag,
        "celeb_img_url": celeb_img_url,
        "view_count": view_count,
        "url": url,
        "production_media_list": []
    }

    production_media_list = []
    production_section_list = soup.find_all('section')

    for production in production_section_list:
        media_div_for_section = production.find_next_sibling()
        production_info = get_production_info(production)

        if media_div_for_section and media_div_for_section.name == 'div':
            media_div_list = media_div_for_section.find_all(
                'div', class_='col-lg-3 col-sm-4 col-xs-6 celebs-boxes albuma')
            media_list = []
            for media in media_div_list:
                media_info = get_media_info(media)
                media_list.append(media_info)
            production_info["media_list"] = media_list

        production_media_list.append(production_info)

    actress_portfolio["production_media_list"] = production_media_list
    return actress_portfolio


def process_page_index(index):
    print("actress page index: ", index)
    # popular_actress_url = f'https://www.aznude.com/browse/celebs/popular/{
    #     index}.html'
    # response = requests.get(popular_actress_url)
    # soup_popular_actress = BeautifulSoup(response.text, 'html.parser')
    # actress_div_list = soup_popular_actress.find_all(
    #     'div', class_="col-lg-2 col-md-3 col-sm-4 col-xs-6 story-thumbs celebs-boxes")
    # actress_portfolio_list = []
    # failed_actress_portfolio_list = []

    # in_page_index = 0

    # for actress in actress_div_list:
    #     actress_url = actress.find('a').get('href')
    #     actress_name = actress.find('h4').text.strip()
    #     try:
    #         view_count = actress.find('span').text.strip()
    #         actress_portfolio = get_actress_portfolio(
    #             url="https://aznude.com" + actress_url, view_count=view_count)
    #         actress_portfolio_list.append(actress_portfolio)
    #         print(actress_name)
    #     except Exception as e:
    #         failed_stat = {
    #             "actress_url": "https://aznude.com" + actress_url,
    #             "actress_name": actress_name,
    #             "in_page_index": in_page_index,
    #             "page_index": index,
    #             "view_count": view_count,
    #             "error": str(e)
    #         }
    #         # print(failed_stat)
    #         failed_actress_portfolio_list.append(failed_stat)
    #         failed_json_string = json.dumps(failed_stat)
    #         with open(f'.\\failed_actress_list\\failed_{index}_{actress_name}.json', 'w') as failed_json_file:
    #             failed_json_file.write(failed_json_string)
    #         print(actress_name, "failed")
    #     in_page_index += 1

    # json_string = json.dumps(actress_portfolio_list)
    # with open(f'.\\actress_portfolio\\actress_portfolio_list_{index}.json', 'w') as json_file:
    #     json_file.write(json_string)

    # return 0


start_index = 51
end_index = 60
num_processes = 2  # You can adjust this value based on your CPU cores
p = Pool(2)

p.map(process_page_index, range(start_index, end_index + 1))